## 1. 환경 설정

`(1) Env 환경변수`

In [ ]:
from dotenv import load_dotenv
load_dotenv()

`(2) 기본 라이브러리`

In [2]:
import os
from glob import glob

from pprint import pprint
import json

## 2. 다양한 문서 형식 처리하기

### 2.1 PDF 문서 

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

pdf_loader = PyPDFLoader('./data/transformer.pdf')
pdf_docs = pdf_loader.load()

len(pdf_docs)

In [ ]:
type(pdf_docs)

In [ ]:
pdf_docs[0]

In [ ]:
print(pdf_docs[0].metadata)

In [ ]:
print(pdf_docs[0].page_content)

### 2.2 웹 문서 

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

web_loader = WebBaseLoader(["https://python.langchain.com/", "https://js.langchain.com/"])

web_docs = web_loader.load()

len(web_docs)


In [ ]:
web_docs[0].metadata

In [ ]:
print(web_docs[0].page_content)

### 2.3 JSON 파일 

In [ ]:
from langchain_community.document_loaders import JSONLoader

json_loader = JSONLoader(
    file_path="./data/kakao_chat.json",
    jq_schema=".messages[].content",    # messages 배열의 content 필드만 추출
    text_content=True,                  # 추출하려는 필드가 텍스트인지 여부
)

json_docs = json_loader.load()

print("문서의 수:", len(json_docs))
print("-" * 50)
print("처음 문서의 메타데이터: \n", json_docs[0].metadata)
print("-" * 50)
print("처음 문서의 내용: \n", json_docs[0].page_content)

In [ ]:
from langchain_community.document_loaders import JSONLoader

json_loader = JSONLoader(
    file_path="./data/kakao_chat.json",
    jq_schema=".messages[]",    # messages 배열의 모든 아이템을 추출
    text_content=False,          # 추출하려는 필드가 텍스트인지 여부
)

json_docs = json_loader.load()

print("문서의 수:", len(json_docs))
print("-" * 50)
print("처음 문서의 메타데이터: \n", json_docs[0].metadata)
print("-" * 50)
print("처음 문서의 내용: \n", json_docs[0].page_content)

In [ ]:
# 유니코드 디코딩 (한글 문자들이 유니코드 이스케이프 시퀀스로 인코딩되어 있음)
from langchain_core.documents import Document

decoded_json_docs = []
for doc in json_docs:

    decoded_data = json.loads(doc.page_content)

    # decoded_json_docs.append({
    #     "metadata": doc.metadata,
    #     "page_content": json.dumps(decoded_data, ensure_ascii=False)
    # })

    document_obj = Document(page_content=json.dumps(decoded_data, ensure_ascii=False), metadata=doc.metadata)
    decoded_json_docs.append(document_obj)

print("문서의 수:", len(decoded_json_docs))
print("-" * 50)
# print("처음 문서의 메타데이터: \n", decoded_json_docs[0]["metadata"])
print("처음 문서의 메타데이터: \n", decoded_json_docs[0].metadata)
print("-" * 50)
# print("처음 문서의 내용: \n", decoded_json_docs[0]["page_content"])
print("처음 문서의 내용: \n", decoded_json_docs[0].page_content)


In [ ]:
# 메타데이터 추가하기
def metadata_func(record: dict, metadata: dict) -> dict:
    metadata["sender"] = record.get("sender")
    metadata["timestamp"] = record.get("timestamp")
    return metadata


json_loader = JSONLoader(
    file_path="./data/kakao_chat.json",
    jq_schema=".messages[]",
    content_key="content",
    metadata_func=metadata_func,
)

json_docs = json_loader.load()

print("문서의 수:", len(json_docs))
print("-" * 50)
print("처음 문서의 메타데이터: \n", json_docs[0].metadata)
print("-" * 50)
print("처음 문서의 내용: \n", json_docs[0].page_content)

```json
{"sender": "김철수", "timestamp": "2023-09-15 09:30:22", "content": "안녕하세요 여러분, 오늘 회의 시간 확인차 연락드립니다."}
{"sender": "이영희", "timestamp": "2023-09-15 09:31:05", "content": "네, 안녕하세요. 오후 2시에 하기로 했어요."}
```

In [ ]:
# JSONL 파일 로드하기
json_loader = JSONLoader(
    file_path="./data/kakao_chat.jsonl",
    jq_schema=".content",
    json_lines=True,
)

json_docs = json_loader.load()

print("문서의 수:", len(json_docs))
print("-" * 50)
print("처음 문서의 메타데이터: \n", json_docs[0].metadata)
print("-" * 50)
print("처음 문서의 내용: \n", json_docs[0].page_content)


In [ ]:
# JSONL 파일 로드하기
json_loader = JSONLoader(
    file_path="./data/kakao_chat.jsonl",
    jq_schema=".",
    content_key="content",
    json_lines=True,
)

json_docs = json_loader.load()

print("문서의 수:", len(json_docs))
print("-" * 50)
print("처음 문서의 메타데이터: \n", json_docs[0].metadata)
print("-" * 50)
print("처음 문서의 내용: \n", json_docs[0].page_content)


In [ ]:
# 메타데이터 추가하기
def metadata_func(record: dict, metadata: dict) -> dict:
    metadata["sender"] = record.get("sender")
    metadata["timestamp"] = record.get("timestamp")
    return metadata

json_loader = JSONLoader(
    file_path="./data/kakao_chat.jsonl",
    jq_schema=".",
    content_key="content",
    metadata_func=metadata_func,
    json_lines=True,
)

json_docs = json_loader.load()

print("문서의 수:", len(json_docs))
print("-" * 50)
print("처음 문서의 메타데이터: \n", json_docs[0].metadata)
print("-" * 50)
print("처음 문서의 내용: \n", json_docs[0].page_content)

### 2.4  CSV 문서 

In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader

csv_loader = CSVLoader("./data/kbo_teams_2023.csv")
csv_docs = csv_loader.load()

print("문서의 수:", len(csv_docs))
print("-" * 50)
print("처음 문서의 메타데이터: \n", csv_docs[0].metadata)
print("-" * 50)
print("처음 문서의 내용: \n", csv_docs[0].page_content)

## 3. 효과적인 텍스트 분할 전략

### 3-1 RecursiveCharacterTextSplitter

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,        # 청크 크기  
    chunk_overlap=200,      # 청크 중 중복되는 부분 크기
    length_function=len,    # 글자 수를 기준으로 분할
    separators=["\n\n", "\n",],  # 구분자
)

texts = text_splitter.split_documents(pdf_docs)
print(f"생성된 텍스트 청크 수: {len(texts)}")
print(f"각 청크의 길이: {list(len(text.page_content) for text in texts)}")

- RecursiveCharacterTextSplitter는 이름에서 알 수 있듯이 재귀적으로 텍스트를 분할합니다. 
- 구분자를 순차적으로 적용하여 큰 청크에서 시작하여 점진적으로 더 작은 단위로 나눕니다. 
- 일반적으로 CharacterTextSplitter보다 더 엄격하게 크기를 준수하려고 합니다.


In [ ]:
# 각 청크의 시작 부분과 끝 부분 확인
for text in texts[:5]:
    print(text.page_content[:200])
    print("-" * 200)
    print(text.page_content[-200:])
    print("=" * 200)
    print()

### 3-2 정규표현식 사용

In [ ]:
# 문장을 구분하여 분할 (마침표, 느낌표, 물음표 다음에 공백이 오는 경우 문장의 끝으로 판단)
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    chunk_size=10,
    chunk_overlap=0,
    separator=r'(?<=[.!?])\s+',
    is_separator_regex=True,
    keep_separator=True,
)

texts = text_splitter.split_documents(json_docs)
print(f"생성된 텍스트 청크 수: {len(texts)}")
print(f"각 청크의 길이: {list(len(text.page_content) for text in texts)}")
print()

# 각 청크의 시작 부분과 끝 부분 확인
for text in texts[:5]:
    print(text.page_content[:50])
    print("-" * 50)
    print(text.page_content[-50:])
    print("=" * 50)
    print()

### 3-3 토큰 수를 기반으로 분할

`(1) tiktoken`  
- OpenAI에서 만든 BPE Tokenizer

In [ ]:
len(pdf_docs[0].page_content)

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name="cl100k_base", 
    # model_name="gpt-4o-mini",
    chunk_size=300, 
    chunk_overlap=0,
)

chunks = text_splitter.split_documents(pdf_docs[:1])

print(f"생성된 청크 수: {len(chunks)}")
print(f"각 청크의 길이: {list(len(chunk.page_content) for chunk in chunks)}")

# 각 청크의 시작 부분과 끝 부분 확인
for chunk in chunks[:5]:
    print(chunk.page_content[:50])
    print("-" * 50)
    print(chunk.page_content[-50:])
    print("=" * 50)
    print()

In [ ]:
import tiktoken

tokenizer = tiktoken.get_encoding("cl100k_base")
# tokenizer = tiktoken.encoding_for_model("gpt-4o-mini")

for chunk in chunks[:5]:

    # 각 청크를 토큰화
    tokens = tokenizer.encode(chunk.page_content)
    # 각 청크의 단어 수 확인
    print(len(tokens))
    # 각 청크의 토큰화 결과 확인 (첫 10개 토큰만 출력)
    print(tokens[:10])
    # 토큰 ID를 실제 토큰(문자열)로 변환해서 출력
    token_strings = [tokenizer.decode([token]) for token in tokens[:10]]
    print(token_strings)

    print("=" * 50)
    print()

`(2) Hugging Face 토크나이저`  
- Hugging Face tokenizer 모델이 토큰 수를 기준으로 분할

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-m3")

tokenizer

In [ ]:
# 토크나이저 인코딩
tokens = tokenizer.encode("안녕하세요. 반갑습니다.")
print(tokens)

In [ ]:
# 토큰을 출력
print(tokenizer.convert_ids_to_tokens(tokens)) 

In [ ]:
# 디코딩
print(tokenizer.decode(tokens))

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer=tokenizer,
    chunk_size=300, 
    chunk_overlap=0,
)

chunks = text_splitter.split_documents(pdf_docs[:1])

print(f"생성된 청크 수: {len(chunks)}")
print(f"각 청크의 길이: {list(len(chunk.page_content) for chunk in chunks)}")
print()

for chunk in chunks[:5]:

    # 각 청크를 토큰화
    tokens = tokenizer.encode(chunk.page_content)
    # 각 청크의 단어 수 확인
    print(len(tokens))
    # 각 청크의 토큰화 결과 확인 (첫 10개 토큰만 출력)
    print(tokens[:10])
    # 토큰 ID를 실제 토큰(문자열)로 변환해서 출력
    token_strings = tokenizer.convert_ids_to_tokens(tokens[:10]) 
    print(token_strings)

    print("=" * 50)
    print()

### 3-4 Semantic Chunking

In [38]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings

# 임베딩 모델을 사용하여 SemanticChunker를 초기화 
text_splitter = SemanticChunker(
    embeddings=OpenAIEmbeddings(),         # OpenAI 임베딩 사용
    breakpoint_threshold_type="gradient",  # 기준점 타입 설정 (gradient, percentile, standard_deviation, interquartile)
    )

In [ ]:
chunks = text_splitter.split_documents(pdf_docs[:1])

print(f"생성된 청크 수: {len(chunks)}")
print(f"각 청크의 길이: {list(len(chunk.page_content) for chunk in chunks)}")
print()


tokenizer = tiktoken.get_encoding("cl100k_base")


for chunk in chunks[:5]:

    # 각 청크를 토큰화

    tokens = tokenizer.encode(chunk.page_content)
    # 각 청크의 단어 수 확인
    print(len(tokens))
    # 각 청크의 내용을 확인
    print(chunk.page_content[:100])

    print("=" * 50)
    print()

## 4. 문서 임베딩(Embedding) 모델

### 4-1 OpenAI 

`(1) embedding 모델`

In [ ]:
from langchain_openai import OpenAIEmbeddings

# OpenAIEmbeddings 모델 생성
embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small")

# 임베딩 객체 출력
embeddings_model

In [ ]:
# 임베딩 모델의 컨텍스트 길이 확인
embeddings_model.embedding_ctx_length

`(2) embed_documents 사용`

In [ ]:
# 문서 컬렉션
documents = [
    "인공지능은 컴퓨터 과학의 한 분야입니다.",
    "머신러닝은 인공지능의 하위 분야입니다.",
    "딥러닝은 머신러닝의 한 종류입니다.",
    "자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.",
    "컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다."
]

# 문서 임베딩
document_embeddings = embeddings_model.embed_documents(documents)

# 임베딩 결과 출력
print(f"임베딩 벡터의 개수: {len(document_embeddings)}")
print(f"임베딩 벡터의 차원: {len(document_embeddings[0])}")
print(document_embeddings[0])

`(3) embed_query 사용`

In [ ]:
embedded_query = embeddings_model.embed_query("인공지능이란 무엇인가요?")

# 쿼리 임베딩 결과 출력
print(f"쿼리 임베딩 벡터의 차원: {len(embedded_query)}")
print(embedded_query)

`(4) 유사도 기반 검색`

In [ ]:
from langchain_community.utils.math import cosine_similarity
import numpy as np

# 쿼리와 가장 유사한 문서 찾기 함수
def find_most_similar(query, doc_embeddings):
    query_embedding = embeddings_model.embed_query(query)
    # similarities = [cosine_similarity(query_embedding, doc_emb) for doc_emb in doc_embeddings]
    similarities = cosine_similarity([query_embedding], doc_embeddings)[0]
    most_similar_idx = np.argmax(similarities)
    return documents[most_similar_idx], similarities[most_similar_idx]

# 예제 쿼리
queries = [
    "인공지능이란 무엇인가요?",
    "딥러닝과 머신러닝의 관계는 어떻게 되나요?",
    "컴퓨터가 이미지를 이해하는 방법은?"
]

# 각 쿼리에 대해 가장 유사한 문서 찾기
for query in queries:
    most_similar_doc, similarity = find_most_similar(query, document_embeddings)
    print(f"쿼리: {query}")
    print(f"가장 유사한 문서: {most_similar_doc}")
    print(f"유사도: {similarity:.4f}")
    print()

### 4-2 Huggingface - 오픈소스 LLM


`(1) embedding 모델`

In [ ]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

# Hugging Face의 임베딩 모델 생성
embeddings_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

# 임베딩 객체 출력
embeddings_model

`(2) embed_documents 사용`

In [ ]:
# 문서 임베딩
document_embeddings = embeddings_model.embed_documents(documents)

# 임베딩 결과 출력
print(f"임베딩 벡터의 개수: {len(document_embeddings)}")
print(f"임베딩 벡터의 차원: {len(document_embeddings[0])}")
print(document_embeddings[0])

`(3) embed_query 사용`

In [ ]:
embedded_query = embeddings_model.embed_query("인공지능이란 무엇인가요?")

# 쿼리 임베딩 결과 출력
print(f"쿼리 임베딩 벡터의 차원: {len(embedded_query)}")
print(embedded_query)

`(4) 유사도 기반 검색`

In [ ]:
# 예제 쿼리
queries = [
    "인공지능이란 무엇인가요?",
    "딥러닝과 머신러닝의 관계는 어떻게 되나요?",
    "컴퓨터가 이미지를 이해하는 방법은?"
]

# 각 쿼리에 대해 가장 유사한 문서 찾기
for query in queries:
    most_similar_doc, similarity = find_most_similar(query, document_embeddings)
    print(f"쿼리: {query}")
    print(f"가장 유사한 문서: {most_similar_doc}")
    print(f"유사도: {similarity:.4f}")
    print()

### 4-3 Ollama - 오픈소스 LLM

`(1) embedding 모델`

In [ ]:
from langchain_ollama import OllamaEmbeddings

# OllamaEmbeddings 모델 생성
# embeddings_model = OllamaEmbeddings(model="nomic-embed-text")
embeddings_model = OllamaEmbeddings(model="bge-m3")

# 임베딩 객체 출력
embeddings_model

`(2) embed_documents 사용`

In [ ]:
# 문서 컬렉션
documents = [
    "인공지능은 컴퓨터 과학의 한 분야입니다.",
    "머신러닝은 인공지능의 하위 분야입니다.",
    "딥러닝은 머신러닝의 한 종류입니다.",
    "자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.",
    "컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다."
]

# 문서 임베딩
document_embeddings = embeddings_model.embed_documents(documents)

# 임베딩 결과 출력
print(f"임베딩 벡터의 개수: {len(document_embeddings)}")
print(f"임베딩 벡터의 차원: {len(document_embeddings[0])}")
print(document_embeddings[0])

`(3) embed_query 사용`

In [ ]:
embedded_query = embeddings_model.embed_query("인공지능이란 무엇인가요?")

# 쿼리 임베딩 결과 출력
print(f"쿼리 임베딩 벡터의 차원: {len(embedded_query)}")
print(embedded_query)

`(4) 유사도 기반 검색`

In [ ]:
# 예제 쿼리
queries = [
    "인공지능이란 무엇인가요?",
    "딥러닝과 머신러닝의 관계는 어떻게 되나요?",
    "컴퓨터가 이미지를 이해하는 방법은?"
]

# 각 쿼리에 대해 가장 유사한 문서 찾기
for query in queries:
    most_similar_doc, similarity = find_most_similar(query, document_embeddings)
    print(f"쿼리: {query}")
    print(f"가장 유사한 문서: {most_similar_doc}")
    print(f"유사도: {similarity:.4f}")
    print()

## 5. 다국어 RAG 시스템 구축 실습

### 5-1 언어 교차(cross-lingual) 검색

`(1) 다국어 문서 로드 및 전처리` 

In [ ]:
korean_txt_files = glob(os.path.join('data', '*_KR.txt')) 
english_txt_files = glob(os.path.join('data', '*_EN.txt'))

print("한국어 텍스트 파일:", korean_txt_files)
print("영어 텍스트 파일:", english_txt_files)

In [ ]:
from langchain_community.document_loaders import TextLoader

def load_text_files(txt_files):
    data = []

    for text_file in txt_files:
        loader = TextLoader(text_file, encoding='utf-8')
        data += loader.load()

    return data


korean_data = load_text_files(korean_txt_files)
english_data = load_text_files(english_txt_files)

print("한국어 데이터 수:", len(korean_data))
print("영어 데이터 수:", len(english_data))

In [ ]:
# 문장을 구분하여 분할 (마침표, 느낌표, 물음표 다음에 공백이 오는 경우 문장의 끝으로 판단)
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    model_name="text-embedding-3-small",
    separator=r"[.!?]\s+",
    chunk_size=100,
    chunk_overlap=0,
    is_separator_regex=True,
    keep_separator=False,
)

korean_docs = text_splitter.split_documents(korean_data)
english_docs = text_splitter.split_documents(english_data)

print("한국어 문서 수:", len(korean_docs))
print("영어 문서 수:", len(english_docs))

In [ ]:
# 각 청크의 시작 부분과 끝 부분 확인
for doc in korean_docs[:2]:
    print(doc.page_content[:50])
    print("-" * 50)
    print(doc.page_content[-50:])
    print("=" * 50)
    print()

In [ ]:
# 각 청크의 시작 부분과 끝 부분 확인
for doc in english_docs[:2]:
    print(doc.page_content[:50])
    print("-" * 50)
    print(doc.page_content[-50:])
    print("=" * 50)
    print()

`(2) 문서 임베딩 및 벡터저장소에 저장 `  

In [63]:
# OpenAI 임베딩 모델 생성
embeddings_openai = OpenAIEmbeddings(model="text-embedding-3-small")

# Hugoing Face 임베딩 모델 생성
embeddings_huggingface = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

# Ollama 임베딩 모델 생성
embeddings_ollama = OllamaEmbeddings(model="nomic-embed-text")

In [ ]:
# 다국어 벡터 저장소 구축
from langchain_chroma import Chroma

db_openai = Chroma.from_documents(
    documents=korean_docs+english_docs, 
    embedding=embeddings_openai,
    collection_name="db_openai",
    persist_directory="./chroma_db",
    )

db_huggingface = Chroma.from_documents(
    documents=korean_docs+english_docs, 
    embedding=embeddings_huggingface,
    collection_name="db_huggingface",
    persist_directory="./chroma_db",
    )

db_ollama = Chroma.from_documents(
    documents=korean_docs+english_docs, 
    embedding=embeddings_ollama,
    collection_name="db_ollama",
    persist_directory="./chroma_db",
    )

# 벡터 저장소에 저장된 문서 수
print(f"OpenAI: {db_openai._collection.count()}")
print(f"Hugging Face: {db_huggingface._collection.count()}")
print(f"Ollama: {db_ollama._collection.count()}")

`(3) RAG 성능 비교 `  

In [65]:
# RAG 체인 생성
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

template = """Answer the question based only on the following context.
Do not use any external information or knowledge. 
If the answer is not in the context, answer "잘 모르겠습니다.".

[Context]
{context}

[Question] 
{question}

[Answer]
"""

prompt = ChatPromptTemplate.from_template(template)


# 문서 포맷터 함수
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


# LLM 모델 생성
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 체인 생성
def create_rag_chain(vectorstore):

    retriever = vectorstore.as_retriever(search_kwargs={'k': 2})

    return (
        {"context": retriever | format_docs , "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

In [66]:
# 체인 생성
rag_chain_openai = create_rag_chain(db_openai)
rag_chain_huggingface = create_rag_chain(db_huggingface)
rag_chain_ollama = create_rag_chain(db_ollama)

In [ ]:
# 한국어 쿼리에 대한 성능 평가
query_ko = "테슬라 창업자는 누구인가요?"

# OpenAI
output_openai = rag_chain_openai.invoke(query_ko)
print("OpenAI:", output_openai)

# Hugging Face
output_huggingface = rag_chain_huggingface.invoke(query_ko)
print("Hugging Face:", output_huggingface)

# Ollama
output_ollama = rag_chain_ollama.invoke(query_ko)
print("Ollama:", output_ollama)

In [ ]:
# 영어 쿼리에 대한 성능 평가
query_en = "Who is the founder of Tesla?"

# OpenAI
output_openai = rag_chain_openai.invoke(query_en)
print("OpenAI:", output_openai)

# Hugging Face
output_huggingface = rag_chain_huggingface.invoke(query_en)
print("Hugging Face:", output_huggingface)

# Ollama
output_ollama = rag_chain_ollama.invoke(query_en)
print("Ollama:", output_ollama)

### 5-2 언어 감지 및 자동번역 통합 

`(1) 한국어 문서 벡터저장소 로드 `  

In [ ]:
# 한국어 문서로 저장되어 있는 벡터 저장소 로드
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small", 
)

vectorstore = Chroma(
    embedding_function=embeddings,
    collection_name="chroma_test",
    persist_directory="./chroma_db",
    )

print(f"벡터 저장소에 저장된 문서 수: {vectorstore._collection.count()}")

In [ ]:
import deepl
from langdetect import detect

translator = deepl.Translator(os.getenv('DEEPL_API_KEY'))

def detect_and_translate(text, target_lang='KO'):
    detected_lang = detect(text)
    if detected_lang.upper() != target_lang:
        result = translator.translate_text(text, target_lang=target_lang)
        return str(result), detected_lang
    return text, detected_lang

# 문서 번역 테스트
text = "Hello. How are you?"
translated_text, detected_lang = detect_and_translate(text, target_lang='KO')
print(f"Detected language: {detected_lang}")
print(f"Translated text: {translated_text}")

`(2) RAG 체인 성능 평가 `  

In [75]:
retriever = vectorstore.as_retriever(search_kwargs={'k': 2})

lang_rag_chain = (
        {"context": retriever | format_docs , "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

# 체인 실행 함수
def run_lang_rag_chain(query):
    original_lang = detect(query)
    
    if original_lang.upper() != 'KO':
        translated_query, _ = detect_and_translate(query, target_lang='KO')
    else:
        translated_query = query
    
    output = lang_rag_chain.invoke(translated_query)
    
    if original_lang.upper() != 'KO':
        target_lang = 'EN-US' if original_lang.upper() == 'EN' else original_lang
        output, _ = detect_and_translate(output, target_lang=target_lang)
    
    return output
    

In [ ]:
# 한국어 쿼리에 대한 성능 평가
query_ko = "테슬라 창업자는 누구인가요?"

output = run_lang_rag_chain(query_ko)
print(output)

# 영어 쿼리에 대한 성능 평가
query_en = "Who is the founder of Tesla?"

output = run_lang_rag_chain(query_en)
print(output)

### 5-3 언어 감지 및 벡터저장소 라우팅

`(1) 언어별 벡터저장소 생성 `  

In [ ]:
# 한국어 문서, 영어 문서를 각각 다른 벡터 저장소에 저장
db_korean = Chroma.from_documents(
    documents=korean_docs, 
    embedding=embeddings_huggingface,    # huggingface 임베딩 사용
    collection_name="db_korean",
    persist_directory="./chroma_db",
    )

db_english = Chroma.from_documents(
    documents=english_docs, 
    embedding=embeddings_ollama,        # ollama 임베딩 사용
    collection_name="db_english",
    persist_directory="./chroma_db",
    )

# 벡터 저장소에 저장된 문서 수
print(f"한국어: {db_korean._collection.count()}")
print(f"영어: {db_english._collection.count()}")

`(2) RAG 체인 성능 평가 `  

In [ ]:
# 체인 실행 함수
from langdetect import detect

rag_chain_korean = create_rag_chain(db_korean)
rag_chain_english = create_rag_chain(db_english)


def run_route_rag_chain(query):
    original_lang = detect(query)
    
    if original_lang.upper() == 'KO':
        return rag_chain_korean.invoke(query)
        
    elif original_lang.upper() == 'EN' :
        return rag_chain_english.invoke(query)
    
    else:
        return "잘 모르겠습니다."
    
# 한국어 쿼리에 대한 성능 평가
query_ko = "테슬라 창업자는 누구인가요?"

output = run_route_rag_chain(query_ko)
print(output)

# 영어 쿼리에 대한 성능 평가
query_en = "Who is the founder of Tesla?"

output = run_route_rag_chain(query_en)
print(output)

## 6. Gradio 챗봇

`(1) invoke 실행` 

- chat history 기능을 추가

In [ ]:
import gradio as gr
from langchain_core.messages import HumanMessage, AIMessage

# 답변 생성 모델을 별도로 사용
answer_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)


def answer_invoke(message, history):
    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))
    history_langchain_format.append(HumanMessage(content=message))

    # 현재 메시지에 대해 RAG 체인 실행
    rag_response = run_route_rag_chain(message)

    # 답변 생성 모델에게 현재 메시지에 대한 답변 요청
    final_answer = answer_llm.invoke(
        history_langchain_format[:-1] + [AIMessage(content=rag_response)] + [HumanMessage(content=message)]
    )
    
    return final_answer.content


# Graiio 인터페이스 생성 
demo = gr.ChatInterface(fn=answer_invoke, title="QA Bot")

# Graiio 실행  
demo.launch()

In [ ]:
# Graiio 종료
demo.close()